In [ ]:
import pandas as pd
import requests

from collections import OrderedDict

!pip install chart_studio
!pip install ipyleaflet

In [ ]:
def get_category_count_from_sparql(sparql_endpoint, query):
    """Carries out the query on the endpoint and returns the value of the counts per category
    :param - sparql_endpoint - the endpoint containing the RDF data to be queried
    :param - query - the query to be carried out. This must select a count variable called ?count and 
    a variable ?category per value of which the count has been calculated
    :returns - a dict of the categories with their counts"""
    category_counts = OrderedDict()
    response = requests.get(
                sparql_endpoint,
                params={"query": query},
                headers = {"Accept": "application/sparql-results+json"})
    for result in response.json()["results"]["bindings"]:
        category_counts[result["category"]["value"]] = int(result["count"]["value"])
        
    return category_counts

## Read the location coordinates

In [ ]:
locations_dataframe = pd.read_csv("https://raw.githubusercontent.com/mwigham/linked_data_notebooks/main/locations.csv")

## Get the location counts

In [ ]:
prefixes = """PREFIX schema: <http://schema.org/>
              PREFIX sdo: <https://schema.org/>
              PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
              PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n"""

sparql_endpoint = "https://cat.apis.beeldengeluid.nl/sparql"
moz_series_id = "<http://data.beeldengeluid.nl/id/series/2101608030025711131>"

In [ ]:
query = prefixes + f"""
            SELECT ?count ?category
            WHERE
            {{
                {{

                    SELECT (COUNT(?category) as ?count) ?category
                    WHERE 
                    {{
                        ?program sdo:partOfSeason/sdo:partOfSeries {moz_series_id}.
                        {{  
                            ?program sdo:locationCreated/sdo:sameAs ?category .
                        }}
                        UNION                
                        {{  
                            ?program sdo:hasPart ?scene.
                            ?scene sdo:locationCreated/sdo:sameAs ?category .
                        }}
                    }} GROUP BY ?category
                }}
            }} ORDER BY DESC(?count) """

distribution = get_category_count_from_sparql(sparql_endpoint, query)

identifiers = []
counts = []
# get the wikidata coordinates for the locations
for location in distribution:
    identifiers.append(location)
    counts.append(distribution[location])

counts_dataframe = pd.DataFrame.from_dict({'Identifiers': identifiers, 
                                             'Counts': counts})

In [ ]:
# combine with the location information
location_counts_dataframe = pd.merge(locations_dataframe, counts_dataframe, on="Identifiers")
print(location_counts_dataframe)

## Plot using Plotly

In [ ]:
import chart_studio
import plotly
from chart_studio import plotly as py
import plotly.graph_objects as go

coords = pd.concat([location_counts_dataframe['Location'],
                    location_counts_dataframe['Latitude'], 
                    location_counts_dataframe['Longitude'], 
                    location_counts_dataframe['Counts']], axis=1)
locations = []
colors = ['rgb(239,243,255)','rgb(189,215,231)','rgb(107,174,214)','rgb(33,113,181)']


locations.append(go.Scattergeo(
    lon = coords['Longitude'],
    lat = coords['Latitude'],
    text = [f"{coords['Location'][i]} {coords['Counts'][i]}" for i in range(len(coords['Location']))],
    marker = dict(
        size = coords['Counts']/60,
        opacity = .4,
        line = dict(width = 0)
    ),
) )

locations[0]['mode'] = 'markers'

                
layout = go.Layout(
    title = 'Standardised concert recording locations in the MOZ collection',
    geo = dict(
        resolution = 110,
        scope = 'europe',
        showframe = False,
        showcoastlines = True,
        showland = True,
        landcolor = "rgb(229, 229, 229)",
        countrycolor = "rgb(255, 255, 255)" ,
        coastlinecolor = "rgb(255, 255, 255)",
        projection = dict(
            type = 'mercator'
        ),
        lonaxis = dict( range= [ 3, 8] ),
        lataxis = dict( range= [ 50,54 ] ),
        
    ),
    legend = dict(
           traceorder = 'reversed'
    ),
    width = 1000,
    height = 800,
)


fig = go.Figure(layout=layout, data=locations)

plotly.offline.iplot(fig, validate=False, filename='maritest')

## Plot using Flourish
Export data and copy into a Flourish map
Example map is available at https://public.flourish.studio/visualisation/14539726/

In [ ]:
location_counts_dataframe.to_csv("location_counts.csv")


## Plot using ipyleaflet

In [ ]:
from ipywidgets import HTML
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(53, 4),
    zoom=6, 
)


for index, row in location_counts_dataframe.iterrows():
    circle_marker = CircleMarker()
    circle_marker.location = (row['Latitude'], row['Longitude'])
    circle_marker.radius = int(row['Counts']/100)
    circle_marker.color = "red"
    circle_marker.fill_color = "red"

    m.add_layer(circle_marker)
    
    # Popup with a given location on the map:
    message = HTML()
    message.value = f"{row['Counts']} concerts"
    message.description = str(row['Location'])

    # Popup associated to a layer
    circle_marker.popup = message

m